# Data Analysis of CEAPS

In [1]:
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display

py.init_notebook_mode(connected=True)

In [2]:
# Chart default layout
default_layout = dict(
    titlefont=dict(size=18, color="darkblue"),
    tickfont=dict(size=14, color="black"),
    showgrid=True,
    zeroline=True,
    showline=True,
    mirror=True,
    gridcolor="lightgrey",
    gridwidth=1,
    zerolinecolor="grey",
    zerolinewidth=2,
    linecolor="black",
    linewidth=2,
)

In [3]:
# Reading the data
original_df = pd.read_csv("../data/interim/data.csv")
# original_df.head()
original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317986 entries, 0 to 317985
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ANO                317986 non-null  int64  
 1   MES                317986 non-null  int64  
 2   SENADOR            317986 non-null  object 
 3   TIPO_DESPESA       317986 non-null  object 
 4   CNPJ_CPF           317986 non-null  object 
 5   FORNECEDOR         317986 non-null  object 
 6   DOCUMENTO          317986 non-null  object 
 7   DATA               314819 non-null  object 
 8   DETALHAMENTO       317986 non-null  object 
 9   VALOR_REEMBOLSADO  317986 non-null  float64
 10  COD_DOCUMENTO      317984 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 26.7+ MB



## Query the data

### Choose the senator

In [4]:
senator = widgets.Dropdown(
    options=original_df["SENADOR"].unique().tolist() + ["All"],
    value="All",
    description="Senator:",
    disabled=False,
)
display(senator)

Dropdown(description='Senator:', index=291, options=('ACIR GURGACZ', 'AÉCIO NEVES', 'ALFREDO NASCIMENTO', 'ALO…

### Choose the supplier

In [5]:
# Grouping the original_df by 'CNPJ_CPF' and finding the most frequent 'FORNECEDOR' in each group
most_frequent_supplier = (
    original_df.groupby("CNPJ_CPF")["FORNECEDOR"]
    .agg(lambda x: x.value_counts().idxmax())
    .reset_index()
)

In [6]:
supplier = widgets.Combobox(
    value="All",
    placeholder="Choose supplier",
    options=most_frequent_supplier["FORNECEDOR"].unique().tolist() + ["All"],
    description="Supplier:",
    ensure_option=True,
    disabled=False,
)

display(supplier)

Combobox(value='All', description='Supplier:', ensure_option=True, options=('Cascol Combustíveis para Veículos…

Obs.: Due to the great amount of suppliers, the search field above group the suppliers by the CNPJ/CPF and show the name that is the most frequent for that CNPJ/CPF. You can also set the search to 'All' to get all the suppliers.

### Choose the year

In [7]:
range_year = widgets.IntRangeSlider(
    value=[2010, 2022],
    min=original_df["ANO"].min(),
    max=original_df["ANO"].max(),
    step=1,
    description="Years",
)
display(range_year)

IntRangeSlider(value=(2010, 2022), description='Years', max=2022, min=2008)

### **After setting the valus above, run the cell below and the others!**

In [8]:
# Querying the data

if senator.value == "All":
    if supplier.value == "All":
        cnpj_cpf = None
        df = original_df[
            original_df["ANO"].isin(range(range_year.value[0], range_year.value[1] + 1))
        ]
    else:
        # Get the CNPJ_CPF of the supplier
        cnpj_cpf = most_frequent_supplier[
            most_frequent_supplier["FORNECEDOR"] == supplier.value
        ]["CNPJ_CPF"].values[0]
        df = original_df[
            (
                original_df["ANO"].isin(
                    range(range_year.value[0], range_year.value[1] + 1)
                )
            )
            & (original_df["CNPJ_CPF"] == cnpj_cpf)
        ]
else:
    if supplier.value == "All":
        cnpj_cpf = None
        df = original_df[
            (
                original_df["ANO"].isin(
                    range(range_year.value[0], range_year.value[1] + 1)
                )
            )
            & (original_df["SENADOR"] == senator.value)
        ]
    else:
        # Get the CNPJ_CPF of the supplier
        cnpj_cpf = most_frequent_supplier[
            most_frequent_supplier["FORNECEDOR"] == supplier.value
        ]["CNPJ_CPF"].values[0]
        df = original_df[
            (
                original_df["ANO"].isin(
                    range(range_year.value[0], range_year.value[1] + 1)
                )
            )
            & (original_df["CNPJ_CPF"] == cnpj_cpf)
            & (original_df["SENADOR"] == senator.value)
        ]
    df = original_df[original_df["SENADOR"] == senator.value]

print(f"Getting data from {range_year.value[0]} to {range_year.value[1]}")
print(f"Senator: {senator.value}")
print(f"Supplier: {supplier.value}", "" if cnpj_cpf is None else f"({cnpj_cpf})")
print(df["ANO"].value_counts().sort_index())

Getting data from 2010 to 2022
Senator: All
Supplier: All 
ANO
2010    18235
2011    25574
2012    30175
2013    29376
2014    22002
2015    26056
2016    26768
2017    27023
2018    24492
2019    21634
2020    14090
2021    16827
2022    16805
Name: count, dtype: int64


## Reibursement by year

In [9]:
# Define x and y axis layout
xaxis_layout = default_layout.copy()
xaxis_layout["title"] = "Year"

yaxis_layout = default_layout.copy()
yaxis_layout["title"] = "Total Reimbursement (R$)"

In [10]:
# Define data
group_by_year = df.groupby("ANO")

# Sum of values by year
reimbursement_by_year = group_by_year["VALOR_REEMBOLSADO"].sum().reset_index()
reimbursement_by_year

,ANO,VALOR_REEMBOLSADO
0,2010,10760010.57
1,2011,19004502.48
2,2012,23354742.37
3,2013,24658952.86
4,2014,22519394.12
5,2015,28893568.61
6,2016,25301959.82
7,2017,26673327.06
8,2018,25604701.33
9,2019,25248242.40


In [11]:
# Plot the data
trace = go.Scatter(
    x=reimbursement_by_year["ANO"],
    y=reimbursement_by_year["VALOR_REEMBOLSADO"],
    mode="markers+lines",
    marker=dict(size=10, color="blue", symbol="circle"),
    line=dict(width=2, color="blue"),
)

# Set chart layout
layout = go.Layout(
    title="Reimbursement by Year",
    titlefont=dict(size=24, color="darkblue"),
    xaxis=xaxis_layout,
    yaxis=yaxis_layout,
    hovermode="closest",
    plot_bgcolor="white",
)

py.iplot({"data": [trace], "layout": layout})

## Distribution of Types of Expenses

In [12]:
# Define x and y axis layout
xaxis_layout = default_layout.copy()
# xaxis_layout["title"] = "Type of expense"

yaxis_layout = default_layout.copy()
yaxis_layout["title"] = "Quantity"

In [13]:
# Define the data
df["TIPO_DESPESA"] = df["TIPO_DESPESA"].astype(str)
count_by_type = df["TIPO_DESPESA"].value_counts().reset_index()
count_by_type.columns = ["TIPO_DESPESA", "COUNT"]
count_by_type

/tmp/ipykernel_349813/1618930082.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,TIPO_DESPESA,COUNT
0,"Locomoçăo, hospedagem, alimentaçăo, combustíve...",122186
1,"Passagens aéreas, aquáticas e terrestres nacio...",68763
2,"Aluguel de imóveis para escritório político, c...",58968
3,Aquisiçăo de material de consumo para uso no e...,23639
4,Divulgaçăo da atividade parlamentar,12722
5,"Contrataçăo de consultorias, assessorias, pesq...",11835
6,Serviços de Segurança Privada,944


In [14]:
# Plot the data
trace = go.Bar(
    x=count_by_type["TIPO_DESPESA"],
    y=count_by_type["COUNT"],
    marker=dict(color="blue"),
    text=count_by_type["COUNT"],
    textposition="auto",
)

# Set chart layout
layout = go.Layout(
    title="Amount of expenses by type",
    titlefont=dict(size=24, color="darkblue"),
    xaxis=dict(
        xaxis_layout,
        showticklabels=False,  # Turn off X axis values
    ),
    yaxis=yaxis_layout,
    hovermode="closest",
    plot_bgcolor="white",
)

py.iplot({"data": [trace], "layout": layout})

## Sum of the expenses by type

In [15]:
# Define x and y axis layout
xaxis_layout = default_layout.copy()
# xaxis_layout["title"] = "Type of expense"

yaxis_layout = default_layout.copy()
yaxis_layout["title"] = "Amount reimbursed (R$)"

In [16]:
# Define the data
amount_by_type = (
    df.groupby("TIPO_DESPESA")["VALOR_REEMBOLSADO"]
    .sum()
    .reset_index()
    .sort_values("VALOR_REEMBOLSADO", ascending=False)
)
amount_by_type

,TIPO_DESPESA,VALOR_REEMBOLSADO
5,"Passagens aéreas, aquáticas e terrestres nacio...",76059445.64
4,"Locomoçăo, hospedagem, alimentaçăo, combustíve...",67702225.63
0,"Aluguel de imóveis para escritório político, c...",55149657.99
2,"Contrataçăo de consultorias, assessorias, pesq...",51983254.59
3,Divulgaçăo da atividade parlamentar,39418784.84
1,Aquisiçăo de material de consumo para uso no e...,11372238.20
6,Serviços de Segurança Privada,3092528.50


In [17]:
# Plot the data
trace = go.Bar(
    x=amount_by_type["TIPO_DESPESA"],
    y=amount_by_type["VALOR_REEMBOLSADO"],
    marker=dict(color="blue"),
    text=amount_by_type["VALOR_REEMBOLSADO"],
    textposition="auto",
)

# Set chart layout
layout = go.Layout(
    title="Amount reimbursed by type of expense",
    titlefont=dict(size=24, color="darkblue"),
    xaxis=dict(
        xaxis_layout,
        showticklabels=False,  # Turn off X axis values
    ),
    yaxis=yaxis_layout,
    hovermode="closest",
    plot_bgcolor="white",
)

py.iplot({"data": [trace], "layout": layout})

## Suppliers of the services (top 20)

In [18]:
# Define x and y axis layout
xaxis_layout = default_layout.copy()
xaxis_layout["title"] = f"Supplier (for senator: {senator.value})"

yaxis_layout = default_layout.copy()
yaxis_layout["title"] = "Amount received (R$)"

In [19]:
# Define the data
amount_by_supplier = (
    df.groupby("FORNECEDOR")["VALOR_REEMBOLSADO"]
    .sum()
    .reset_index()
    .sort_values("VALOR_REEMBOLSADO", ascending=False)
)
amount_by_supplier[:20]

,FORNECEDOR,VALOR_REEMBOLSADO
26018,LATAM,5276803.89
19893,GOL,4652758.73
45402,TAM,4507334.20
35712,PROPAGTUR - Propag Turismo Ltda,4442760.27
936,ADRIA VIAGENS E TURISMO LTDA,3450669.70
12883,Citel Service,1634467.13
24960,Jefferson L R Coronel - Me,1520200.00
934,ADRIA VIAGENS E TURISMO,1513759.85
4331,Adria Viagens e Turismo Ltda,1473108.67
3914,AVIANCA,1363992.97


In [20]:
# Plot the data
trace = go.Bar(
    x=amount_by_supplier["FORNECEDOR"][:20],
    y=amount_by_supplier["VALOR_REEMBOLSADO"][:20],
    marker=dict(color="blue"),
    text=amount_by_supplier["VALOR_REEMBOLSADO"][:20],
    textposition="auto",
)

# Set chart layout
layout = go.Layout(
    title="Amount reimbursed by supplier",
    titlefont=dict(size=24, color="darkblue"),
    xaxis=dict(
        xaxis_layout,
        showticklabels=False,  # Turn off X axis values
    ),
    yaxis=yaxis_layout,
    hovermode="closest",
    plot_bgcolor="white",
)

py.iplot({"data": [trace], "layout": layout})

## Senator for the supplier (top 20)

In [21]:
# Define x and y axis layout
xaxis_layout = default_layout.copy()
xaxis_layout["title"] = f"Senator (for  supplier: {supplier.value})"

yaxis_layout = default_layout.copy()
yaxis_layout["title"] = "Amount paid (R$)"

In [22]:
# Define the data
amount_by_senator = (
    df.groupby("SENADOR")["VALOR_REEMBOLSADO"]
    .sum()
    .reset_index()
    .sort_values("VALOR_REEMBOLSADO", ascending=False)
)
amount_by_senator[:20]

,SENADOR,VALOR_REEMBOLSADO
61,EDUARDO AMORIM,6228794.68
255,SÉRGIO PETECĂO,5088726.58
80,FERNANDO COLLOR,4937215.19
105,HUMBERTO COSTA,4663461.97
39,CIRO NOGUEIRA,4579717.29
218,RANDOLFE RODRIGUES,4571437.31
0,ACIR GURGACZ,4504657.59
207,PAULO PAIM,4223362.06
63,EDUARDO BRAGA,3981268.98
266,VANESSA GRAZZIOTIN,3870627.57


In [23]:
# Plot the data
trace = go.Bar(
    x=amount_by_senator["SENADOR"][:20],
    y=amount_by_senator["VALOR_REEMBOLSADO"][:20],
    marker=dict(color="blue"),
    text=amount_by_senator["VALOR_REEMBOLSADO"],
    textposition="auto",
)

# Set chart layout
layout = go.Layout(
    title="Amount reimbursed by senator",
    titlefont=dict(size=24, color="darkblue"),
    xaxis=dict(
        xaxis_layout,
        showticklabels=False,  # Turn off X axis values
    ),
    yaxis=yaxis_layout,
    hovermode="closest",
    plot_bgcolor="white",
)

py.iplot({"data": [trace], "layout": layout})